In [ ]:
import requests
from datetime import datetime
from matplotlib import pyplot as plt
from time import sleep

In [ ]:
def getKillMails():
    """
    get all the kills of the AKINA mountain family corporation from https://zkillboard.com/corporation/98615098/kills/
    this is the corporation that hosts all the CCTV EYES XY, AKINA MOUNTAIN XY etc.
    """
    morePages = True
    page = 1
    killmails = []

    while morePages == True:
        url = "https://zkillboard.com/api/kills/corporationID/98615098/page/" + str(page) + "/"
        
        response = requests.get(url)
        

        #zkill does not seem to like getting DDoSed, so adding a delay
        sleep(1)
        

        if response.status_code != 200:
            morePages = False
        
        else:
            killmails = killmails + response.json()
            page += 1

            #only take the first 30 pages = 6000 killmails
            if page > 30:
                morePages = False

        
    return(killmails)

def getKillMailDetails(id, hash):
    """
    the zkillboard response does not include many details, so retrieve those from ESI
    """

    url = "https://esi.evetech.net/latest/killmails/" + str(id) + "/" + str(hash) + "/?datasource=tranquility"
    response = requests.get(url)

    return(response.json())


In [ ]:
#get the killmails
killmails = getKillMails()
print(len(killmails))

In [ ]:
killmailDetails = []

# this will take a while, expect 4-5 minutes per 1000 killmails. go get yourself a coffee.
# it will also likely timeout at some point as ESI does not like to get DDoSed either, but I do not care enough to add a delay here

for km in killmails:
    killmailID = km["killmail_id"]
    killmailHash = km["zkb"]["hash"]

    details = getKillMailDetails(killmailID, killmailHash)

    killmailDetails.append(details)

print(len(killmailDetails),"killmails in total")
    
    

In [ ]:
print(killmailDetails[3])

In [ ]:
#ignore large fights
prunedKillmails = []

for km in killmailDetails:
    killmailAdded = False

    try:
        if km["victim"]["ship_type_id"] != 670:                                                         # ignore pods

            try:
                if len(km["attackers"]) < 9:                                                            # this is to limit the killmails to (mostly)drops, rather than large fleet fights, that the members of AKINA mountain family join
                    # if (datetime.now() - datetime.fromisoformat(km["killmail_time"][:-1])).days > 90:   # killmails from last 90 days
                    for attacker in km["attackers"]:
                        if attacker["ship_type_id"] == 11969 and killmailAdded == False:            # add killmails including an arazu to the list
                            prunedKillmails.append(km)
                            killmailAdded = True

            
            except: pass
    except: pass
print(len(prunedKillmails), "killmails in total")

In [ ]:
finalBlowAlliance = []
suspiciousKillmails = []
for km in prunedKillmails:
    for attacker in km["attackers"]:
        if attacker["final_blow"]:
            try:
                if attacker["alliance_id"] != 99011168:                                                 # add kills where the killing blow was not dealt by member of AKINA mountain family to separate list
                    suspiciousKillmails.append(km)
                    finalBlowAlliance.append(attacker["alliance_id"])

                    
            except: pass

In [ ]:
# this can be ignored now

print(len(suspiciousKillmails))
print(len(set(finalBlowAlliance)))

In [ ]:
killmailTimes = []
for km in prunedKillmails:
    killmailTimes.append(km["killmail_time"])

killHour = []

for time in killmailTimes:
    killHour.append(datetime.fromisoformat(time[:-1]).hour)

In [ ]:
# this can be ignored now

suspiciousHours = []
suspiciousTimes = []

for km in suspiciousKillmails:
    suspiciousTimes.append(km["killmail_time"])

for time in suspiciousTimes:
    suspiciousHours.append(datetime.fromisoformat(time[:-1]).hour)

In [ ]:
# this can be ignored now

plt.figure(figsize=(24, 12))

plt.hist(killHour, bins=np.arange(24)-0.5, rwidth=0.9)
plt.hist(suspiciousHours, bins=np.arange(24)-0.5, rwidth=0.9)
plt.xlabel('hour of day')
plt.xticks(np.arange(1,24,1))
plt.yticks(np.arange(1,60,5))
plt.ylabel('Number of kills')
plt.grid()

In [ ]:
# this can be ignored now

t0 = prunedKillmails[0]["killmail_time"][:-1]
timeDeltas = []
timeOfKill = []

for km in prunedKillmails[1:]:

    timeDeltas.append(datetime.fromisoformat(t0) - datetime.fromisoformat(km["killmail_time"][:-1]))
    timeOfKill.append(datetime.fromisoformat(km["killmail_time"][:-1]))
    
    t0 = km["killmail_time"][:-1]


In [ ]:
# this can be ignored now


timeDeltasHours = []
sleep = []

for delta in timeDeltas:
    hours = delta.seconds / 3600
    timeDeltasHours.append(hours)
    if hours > 6 and hours <24:
        sleep.append(hours)
        

In [ ]:
# this can be ignored now

print("average between kills ",np.average(timeDeltasHours))
print("median time between kills ",np.median(timeDeltasHours))
print("max time between kills ", np.max(timeDeltasHours))


In [ ]:
# create a separate old killmails as baseline to compare the new ones against

olderKillmails = []
newerKillmails = []

for km in prunedKillmails:
    if (datetime.now() - datetime.fromisoformat(km["killmail_time"][:-1])).days > 5:   # killmails from last 90 days
        olderKillmails.append(km)

for km in prunedKillmails:
    if (datetime.now() - datetime.fromisoformat(km["killmail_time"][:-1])).days <= 5:   # killmails from last 90 days
        newerKillmails.append(km)

In [ ]:
newKillmailTimes = []
oldKillmailTimes = []

for km in olderKillmails:
    oldKillmailTimes.append(datetime.fromisoformat(km["killmail_time"][:-1]).hour)

for km in newerKillmails:
    newKillmailTimes.append(datetime.fromisoformat(km["killmail_time"][:-1]).hour)


In [ ]:
plt.figure(figsize=(24, 12))

plt.hist(oldKillmailTimes, bins=np.arange(24)-0.5, rwidth=0.9)
plt.hist(newKillmailTimes, bins=np.arange(24)-0.5, rwidth=0.9)
plt.xlabel('hour of day')
plt.xticks(np.arange(1,24,1))
plt.yticks(np.arange(0,80,2))
plt.ylabel('Number of kills')
plt.grid()
